In [333]:
import pandas as pd
import csv
import numpy as np
from pathlib import Path
from itertools import product
from tkinter import *
import tkinter as tk
from tkinter import ttk



from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
%matplotlib inline


ModuleNotFoundError: No module named 'Tkinter'

In [2]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'PHX'][0]
celtics_id = celtics['id']

from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2020':]

# Subset the games to when the last 4 digits of SEASON_ID were 2017.
games_2021 = games[games.SEASON_ID.str[-4:] == '2020']
games_2021.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42020,1610612756,PHX,Phoenix Suns,0042000313,2021-06-24,PHX @ LAC,L,240,92,...,0.923,11,32,43,24,5,3,10,19,-14.0
1,42020,1610612756,PHX,Phoenix Suns,0042000312,2021-06-22,PHX vs. LAC,W,239,104,...,0.889,7,32,39,24,5,3,9,20,1.0
2,42020,1610612756,PHX,Phoenix Suns,0042000311,2021-06-20,PHX vs. LAC,W,240,120,...,1.000,7,36,43,31,5,3,7,16,6.0
3,42020,1610612756,PHX,Phoenix Suns,0042000234,2021-06-13,PHX @ DEN,W,241,125,...,0.935,7,41,48,18,6,6,14,21,7.0
4,42020,1610612756,PHX,Phoenix Suns,0042000233,2021-06-11,PHX @ DEN,W,239,116,...,0.900,4,34,38,26,10,2,12,13,14.0


In [3]:
from basketball_reference_scraper.seasons import get_schedule

In [4]:
playoff2020 = get_schedule(2020, playoffs = True)

In [5]:
playoff2020

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
1060,2020-08-17,Utah Jazz,125,Denver Nuggets,135
1061,2020-08-17,Brooklyn Nets,110,Toronto Raptors,134
1062,2020-08-17,Philadelphia 76ers,101,Boston Celtics,109
1063,2020-08-17,Dallas Mavericks,110,Los Angeles Clippers,118
1064,2020-08-18,Orlando Magic,122,Milwaukee Bucks,110
...,...,...,...,...,...
1138,2020-10-02,Miami Heat,114,Los Angeles Lakers,124
1139,2020-10-04,Los Angeles Lakers,104,Miami Heat,115
1140,2020-10-06,Los Angeles Lakers,102,Miami Heat,96
1141,2020-10-09,Miami Heat,111,Los Angeles Lakers,108


In [30]:
playoff2019 = get_schedule(2019, playoffs = True)
playoff2019
np.savetxt('2019playoff.csv', playoff2019, delimiter =",", fmt ="% s")

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
1231,2019-04-13,Brooklyn Nets,111,Philadelphia 76ers,102
1232,2019-04-13,Orlando Magic,104,Toronto Raptors,101
1233,2019-04-13,Los Angeles Clippers,104,Golden State Warriors,121
1234,2019-04-13,San Antonio Spurs,101,Denver Nuggets,96
1235,2019-04-14,Indiana Pacers,74,Boston Celtics,84
...,...,...,...,...,...
1308,2019-06-02,Golden State Warriors,109,Toronto Raptors,104
1309,2019-06-05,Toronto Raptors,123,Golden State Warriors,109
1310,2019-06-07,Toronto Raptors,105,Golden State Warriors,92
1311,2019-06-10,Golden State Warriors,106,Toronto Raptors,105


In [6]:
np.savetxt('2020playoff.csv', playoff2020, delimiter =",", fmt ="% s")

In [126]:
csv_path = Path('2020data.csv')
df_2020 = pd.read_csv(csv_path)
df_2020.head()

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,10.08,32.56,42.64,25.37,7.19,4.44,14.26,20.47,109.18,160
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,10.92,35.14,46.06,25.57,7.22,5.49,16.83,20.36,106.51,450
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,10.32,34.01,44.33,22.40,7.14,5.50,15.24,20.65,107.35,230
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,10.71,34.60,45.31,23.74,7.50,4.69,14.07,22.88,109.90,285
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,9.49,36.32,45.81,24.16,7.37,4.47,14.11,21.73,108.59,900


In [127]:
df_2020['VSS'] = '0'
df_2020['HSS'] = '0'
df_2020

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,VSS,HSS
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,42.64,25.37,7.19,4.44,14.26,20.47,109.18,160,0,0
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,46.06,25.57,7.22,5.49,16.83,20.36,106.51,450,0,0
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,44.33,22.40,7.14,5.50,15.24,20.65,107.35,230,0,0
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,45.31,23.74,7.50,4.69,14.07,22.88,109.90,285,0,0
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,45.81,24.16,7.37,4.47,14.11,21.73,108.59,900,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,(2019-20),10/2/2020 0:00,MIA,LAL,114,124,-10,39.5,84.4,0.47,...,42.27,23.41,8.24,3.70,15.92,21.75,107.65,450,0,0
79,(2019-20),10/4/2020 0:00,LAL,MIA,104,115,-11,42.3,88.3,0.48,...,42.53,24.58,7.47,4.14,14.00,21.70,109.10,-420,0,0
80,(2019-20),10/6/2020 0:00,LAL,MIA,102,96,6,42.3,88.3,0.48,...,42.53,24.58,7.47,4.14,14.00,21.70,109.10,-330,0,0
81,(2019-20),10/9/2020 0:00,MIA,LAL,111,108,3,39.5,84.4,0.47,...,42.27,23.41,8.24,3.70,15.92,21.75,107.65,280,0,0


## def hss(df_2020):
    if df_2020['Mar.'] <0:
        return '1'
    else:
        return '0'
## df_2020['HSS'] = df_2020.apply(hss,axis=1)

## def vss(df_2020):
    if df_2020['Mar.'] >0:
        return '1'
    else:
        return '0'
## df_2020['VSS'] = df_2020.apply(vss,axis=1)
## df_2020.head(5)

In [128]:
df_2020['matches'] = df_2020['VTeam']+df_2020['Hteam']
u_id = []
for i, r in df_2020.iterrows():
    if r.loc['matches'][0:3] < r.loc['matches'][3:7]:
        u_id_ = r.matches[0:3] + r.matches[3:7]
        u_id.append(u_id_)
    elif r.loc['matches'][0:3] > r.loc['matches'][3:7]:
        u_id_ = r.matches[3:7] + r.matches[0:3]
        u_id.append(u_id_)
#         print(False, r['u_id'])
df_2020['u_id'] = u_id
df_2020


,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,VSS,HSS,matches,u_id
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,7.19,4.44,14.26,20.47,109.18,160,0,0,UTADEN,DENUTA
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,7.22,5.49,16.83,20.36,106.51,450,0,0,BROTOR,BROTOR
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,7.14,5.50,15.24,20.65,107.35,230,0,0,PHIBOS,BOSPHI
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,7.50,4.69,14.07,22.88,109.90,285,0,0,DALLAC,DALLAC
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,7.37,4.47,14.11,21.73,108.59,900,0,0,ORLMIL,MILORL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,(2019-20),10/2/2020 0:00,MIA,LAL,114,124,-10,39.5,84.4,0.47,...,8.24,3.70,15.92,21.75,107.65,450,0,0,MIALAL,LALMIA
79,(2019-20),10/4/2020 0:00,LAL,MIA,104,115,-11,42.3,88.3,0.48,...,7.47,4.14,14.00,21.70,109.10,-420,0,0,LALMIA,LALMIA
80,(2019-20),10/6/2020 0:00,LAL,MIA,102,96,6,42.3,88.3,0.48,...,7.47,4.14,14.00,21.70,109.10,-330,0,0,LALMIA,LALMIA
81,(2019-20),10/9/2020 0:00,MIA,LAL,111,108,3,39.5,84.4,0.47,...,8.24,3.70,15.92,21.75,107.65,280,0,0,MIALAL,LALMIA


In [130]:
def ss(df_2020):
    if df_2020['Mar.']<0 :
        return df_2020['Hteam']
    else:
        return df_2020['VTeam']
    
df_2020['winner'] = df_2020.apply(ss,axis=1)
df_2020.head(20)


,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,VSS,HSS,matches,u_id,winner
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,4.44,14.26,20.47,109.18,160,0,0,UTADEN,DENUTA,DEN
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,5.49,16.83,20.36,106.51,450,0,0,BROTOR,BROTOR,TOR
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,5.50,15.24,20.65,107.35,230,0,0,PHIBOS,BOSPHI,BOS
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,4.69,14.07,22.88,109.90,285,0,0,DALLAC,DALLAC,LAC
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,4.47,14.11,21.73,108.59,900,0,0,ORLMIL,MILORL,ORL
5,(2019-20),8/18/2020 0:00,MIA,IND,113,101,12,39.5,84.4,0.47,...,4.59,14.67,18.81,107.48,-185,0,0,MIAIND,INDMIA,MIA
6,(2019-20),8/18/2020 0:00,OCT,HOU,108,123,-15,40.0,85.5,0.47,...,4.92,16.15,21.04,114.85,-110,0,0,OCTHOU,HOUOCT,HOU
7,(2019-20),8/18/2020 0:00,POR,LAL,100,93,7,42.1,91.1,0.46,...,3.70,15.92,21.75,107.65,250,0,0,PORLAL,LALPOR,POR
8,(2019-20),8/19/2020 0:00,PHI,BOS,101,128,-27,41.1,87.9,0.47,...,5.50,15.24,20.65,107.35,175,0,0,PHIBOS,BOSPHI,BOS
9,(2019-20),8/19/2020 0:00,DAL,LAC,127,114,13,41.7,90.3,0.46,...,4.69,14.07,22.88,109.90,175,0,0,DALLAC,DALLAC,DAL


In [73]:
df_2020['series_id'] = df_2020['Season']+ df_2020['VTeam'] + df_2020['Hteam']

df_2020

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,VSS,HSS,series_id
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,25.37,7.19,4.44,14.26,20.47,109.18,160,0,1,(2019-20)UTADEN
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,25.57,7.22,5.49,16.83,20.36,106.51,450,0,1,(2019-20)BROTOR
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,22.40,7.14,5.50,15.24,20.65,107.35,230,0,1,(2019-20)PHIBOS
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,23.74,7.50,4.69,14.07,22.88,109.90,285,0,1,(2019-20)DALLAC
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,24.16,7.37,4.47,14.11,21.73,108.59,900,1,0,(2019-20)ORLMIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,(2019-20),10/2/2020 0:00,MIA,LAL,114,124,-10,39.5,84.4,0.47,...,23.41,8.24,3.70,15.92,21.75,107.65,450,0,1,(2019-20)MIALAL
79,(2019-20),10/4/2020 0:00,LAL,MIA,104,115,-11,42.3,88.3,0.48,...,24.58,7.47,4.14,14.00,21.70,109.10,-420,0,1,(2019-20)LALMIA
80,(2019-20),10/6/2020 0:00,LAL,MIA,102,96,6,42.3,88.3,0.48,...,24.58,7.47,4.14,14.00,21.70,109.10,-330,1,0,(2019-20)LALMIA
81,(2019-20),10/9/2020 0:00,MIA,LAL,111,108,3,39.5,84.4,0.47,...,23.41,8.24,3.70,15.92,21.75,107.65,280,1,0,(2019-20)MIALAL


In [8]:
df_2020.set_index('Time', inplace=True)
df_2020

,Season,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,3FGM,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
Time,,,,,,,,,,,,,,,,,,,,,
8/17/2020 0:00,(2019-20),UTA,DEN,125,135,-10,40.1,85.1,0.47,13.4,...,10.08,32.56,42.64,25.37,7.19,4.44,14.26,20.47,109.18,160
8/17/2020 0:00,(2019-20),BRO,TOR,110,134,-24,40.4,90.3,0.45,13.1,...,10.92,35.14,46.06,25.57,7.22,5.49,16.83,20.36,106.51,450
8/17/2020 0:00,(2019-20),PHI,BOS,101,109,-8,41.1,87.9,0.47,11.6,...,10.32,34.01,44.33,22.40,7.14,5.50,15.24,20.65,107.35,230
8/17/2020 0:00,(2019-20),DAL,LAC,110,118,-8,41.7,90.3,0.46,15.1,...,10.71,34.60,45.31,23.74,7.50,4.69,14.07,22.88,109.90,285
8/18/2020 0:00,(2019-20),ORL,MIL,122,110,12,39.3,88.6,0.44,11.1,...,9.49,36.32,45.81,24.16,7.37,4.47,14.11,21.73,108.59,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10/2/2020 0:00,(2019-20),MIA,LAL,114,124,-10,39.5,84.4,0.47,13.4,...,9.41,32.86,42.27,23.41,8.24,3.70,15.92,21.75,107.65,450
10/4/2020 0:00,(2019-20),LAL,MIA,104,115,-11,42.3,88.3,0.48,11.0,...,9.23,33.30,42.53,24.58,7.47,4.14,14.00,21.70,109.10,-420
10/6/2020 0:00,(2019-20),LAL,MIA,102,96,6,42.3,88.3,0.48,11.0,...,9.23,33.30,42.53,24.58,7.47,4.14,14.00,21.70,109.10,-330


In [9]:
# Set x variable list of features 
df_2020_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line'
                     ]

In [301]:
# Construct the X_train and y_train datasets
df_2020_x_train = df_2020[df_2020_x_var_list]
df_2020_y_train = df_2020['Mar.']

In [13]:
csv_path = Path('2021playoff.csv')
df_2021 = pd.read_csv(csv_path)
df_2021.head()

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,(2020-21),5/22/2021 0:00,MIA,MIL,107.0,109.0,-2,39.22,83.74,0.47,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
1,(2020-21),5/22/2021 0:00,DAL,LAC,113.0,103.0,10,41.08,87.32,0.47,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
2,(2020-21),5/22/2021 0:00,BOS,BRO,93.0,104.0,-11,41.46,88.90,0.47,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
3,(2020-21),5/22/2021 0:00,POR,DEN,123.0,109.0,14,41.29,91.08,0.45,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
4,(2020-21),5/23/2021 0:00,WAS,PHI,118.0,125.0,-7,43.17,90.93,0.47,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0


In [14]:
df_2021.set_index('Time', inplace=True)
df_2021

,Season,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,3FGM,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
Time,,,,,,,,,,,,,,,,,,,,,
5/22/2021 0:00,(2020-21),MIA,MIL,107.0,109.0,-2,39.22,83.74,0.47,12.94,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
5/22/2021 0:00,(2020-21),DAL,LAC,113.0,103.0,10,41.08,87.32,0.47,13.81,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
5/22/2021 0:00,(2020-21),BOS,BRO,93.0,104.0,-11,41.46,88.90,0.47,13.60,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
5/22/2021 0:00,(2020-21),POR,DEN,123.0,109.0,14,41.29,91.08,0.45,15.72,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
5/23/2021 0:00,(2020-21),WAS,PHI,118.0,125.0,-7,43.17,90.93,0.47,10.18,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6/30/2021 0:00,(2020-21),PHO,LAC,NaN,NaN,0,43.31,88.29,0.49,13.06,...,8.75,34.19,42.94,26.93,7.18,4.33,12.53,19.08,115.28,NaN
7/1/2021 0:00,(2020-21),ATL,MIL,NaN,NaN,0,40.79,87.24,0.47,12.43,...,10.56,35.07,45.63,24.13,6.99,4.75,13.24,19.33,113.69,NaN
7/2/2021 0:00,(2020-21),LAC,PHO,NaN,NaN,0,41.79,86.69,0.48,14.26,...,9.42,34.74,44.15,24.39,7.07,4.10,13.19,19.21,114.01,NaN


In [15]:
df_2021_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line'
                     ]

In [18]:
playoff2021_testing_start =  '5/22/2021 0:00'
playoff2021_testing_end = '6/20/2021 0:00'

In [306]:
df_2021_test_X = df_2021[df_2021_x_var_list][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_X = df_2021_test_X.reset_index().drop(columns = 'Time')
df_2021_test_y = df_2021['Mar.'][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_X

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,39.22,83.74,0.47,12.94,36.19,0.36,16.68,21.11,0.79,8.04,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
1,41.08,87.32,0.47,13.81,38.11,0.36,16.47,21.17,0.78,9.13,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
2,41.46,88.90,0.47,13.60,36.36,0.37,16.11,20.78,0.78,10.63,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
3,41.29,91.08,0.45,15.72,40.82,0.39,17.81,21.64,0.82,10.64,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
4,43.17,90.93,0.47,10.18,29.00,0.35,20.13,26.17,0.77,9.68,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,41.40,86.90,0.48,11.26,30.13,0.37,19.57,25.50,0.77,10.03,...,10.03,35.03,45.06,23.69,9.10,6.21,14.44,20.22,113.64,175.0
64,41.26,88.11,0.47,16.74,43.03,0.39,17.15,21.47,0.80,10.63,...,10.63,37.63,48.25,23.65,6.58,5.15,14.21,18.54,116.42,-155.0
65,44.74,91.81,0.49,14.42,37.07,0.39,16.24,21.38,0.76,10.29,...,10.29,37.83,48.13,25.47,8.13,4.64,13.82,17.28,120.13,-145.0
66,41.79,86.69,0.48,14.26,34.69,0.41,16.17,19.26,0.84,9.42,...,9.42,34.74,44.15,24.39,7.07,4.10,13.19,19.21,114.01,150.0


In [307]:
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_2020_forest_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
df_2020_forest_model.fit(df_2020_x_train, df_2020_y_train)



RandomForestClassifier(max_depth=3, random_state=0)

In [308]:
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions = df_2020_forest_model.predict(df_2021_test_X)


In [309]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results = df_2021_test_y.to_frame()
playoff2021_results["Predicted Mar."] = playoff2021_predictions
playoff2021_results.head(20)

,Mar.,Predicted Mar.
Time,,
5/22/2021 0:00,-2,-10
5/22/2021 0:00,10,-2
5/22/2021 0:00,-11,-2
5/22/2021 0:00,14,-12
5/23/2021 0:00,-7,8
5/23/2021 0:00,-9,-8
5/23/2021 0:00,2,8
5/23/2021 0:00,3,-2
5/24/2021 0:00,-34,-10


In [261]:
# Save the pre-trained model
from joblib import dump, load
dump(df_2020_forest_model, 'predictor.joblib')

['predictor.joblib']

## predictor

In [243]:
teamstats2021_path = Path('2021teamseasonstats.csv')
df_team_2021 = pd.read_csv(teamstats2021_path)
df_team_2021 = df_team_2021.set_index('Team')
df_team_2021.head()

,League,Type,Season,Team/Opponent,Champion,Franchise,FGM,FGA,FG%,3FGM,...,OFF,DEF,REB,AST,STL,BLK,TO,PF,PTS,Unnamed: 25
Team,,,,,,,,,,,,,,,,,,,,,
ATL,NBA,Regular Season,(2020-21),Team Stats,NaN,Atlanta Hawks,40.8,87.2,0.47,12.4,...,10.6,35.1,45.6,24.1,7.0,4.8,13.2,19.3,113.7,NaN
BOS,NBA,Regular Season,(2020-21),Team Stats,NaN,Boston Celtics,41.5,88.9,0.47,13.6,...,10.6,33.6,44.3,23.5,7.7,5.3,14.1,20.4,112.6,NaN
BRO,NBA,Regular Season,(2020-21),Team Stats,NaN,Brooklyn Nets,43.1,87.3,0.49,14.2,...,8.9,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6,NaN
CHA,NBA,Regular Season,(2020-21),Team Stats,NaN,Charlotte Hornets,39.9,87.8,0.45,13.7,...,10.6,33.2,43.8,26.8,7.8,4.8,14.8,18.0,109.5,NaN
CHI,NBA,Regular Season,(2020-21),Team Stats,NaN,Chicago Bulls,42.2,88.6,0.48,12.6,...,9.6,35.3,45.0,26.8,6.7,4.2,15.1,18.9,110.7,NaN


In [119]:
# creater a predictor df 
df_converter_columns = ['HTeam','VTeam','Betting Line','FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS'
                     ]
df_converter = pd.DataFrame(columns=df_converter_columns)
df_converter = df_converter.fillna(0) 
df_converter

,HTeam,VTeam,Betting Line,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,...,OPFT%,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS


In [ ]:
def pred():
    

In [332]:
# Create UI for our prediction program
root = Tk()
root.title('NBA Predictor')
root.geometry("555x350")

head = Label(root, text = "2020-21 NBA Playoffs Game Result Predictor", font = ('comic sans',20),
            fg = 'blue', relief = tk.RAISED, borderwidth = 3)
head.grid(row=0,column=0,columnspan=3)

team = Label(root, text = "Away Team     @     Home Team", font = ('comic sans',20))
team.grid(row=1,column=0,columnspan=2)             
             
hometeam = ttk.Combobox(root,font = ('comic sans',20),width=8)
hometeam['value'] = ('ATL','BKN','BOS','DAL','DEN','LAC','LAL','MEM',
                     'MIA','MIL','NYK','PHI','PHX','POR','UTA','WAS')
hometeam.grid(row=2,column=0,padx=5,pady=5,columnspan=1)



awayteam = ttk.Combobox(root,font = ('comic sans',20),width=8)
awayteam['value'] = ('ATL','BKN','BOS','DAL','DEN','LAC','LAL','MEM',
                     'MIA','MIL','NYK','PHI','PHX','POR','UTA','WAS')
awayteam.grid(row=2,column=1,padx=5,pady=5,columnspan=1)

ssbl = Label(root, text = "Series Score          Money Line", font = ('comic sans',20))
ssbl.grid(row=3,column=0,columnspan=2) 
seriesscore = ttk.Combobox(root,font = ('comic sans',20),width=8)
seriesscore['value'] = ('0-0','0-1','0-2','0-3','1-0','1-1','1-2','1-3',
                     '2-0','2-1','2-2','2-3','3-0','3-1','3-2','3-3')
seriesscore.grid(row=4,column=0,padx=5,pady=5,columnspan=1)

userin = IntVar()
bettingline= Entry(root,textvariable=userin,font = ('comic sans',20),width=9)
bettingline.grid(row=4,column=1,padx=5,pady=5,columnspan=1)
predictor_df = pd.DataFrame()

predictor_df = pd.DataFrame()

def action():
    global DB
    import pandas as pd
    predictor_df = pd.DataFrame(columns=['HTeam','VTeam','Betting Line','FGM','FGA','FG%','3FGM',
                                         '3FGA','3FG%','FTM','FTA','FT%','OFF','DEF','REB','AST',
                                         'STL','BLK','TO','PF','PTS','OPFGM','OPFGA','OPFG%',
                                         'OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%','OPOFF',
                                         'OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF','OPPTS'])
    #predictor_df.loc[0,'HTeam']=hometeam
    #predictor_df.loc[0,'VTeam']=awayteam
    #predictor_df.loc[0,'Betting Line']=bettingline
    
    FGM=df_team_2021.loc[hometeam,'FGM']
    predictor_df.loc[0,'FGM']=FGM
    
    FGA=df_team_2021.loc[hometeam,'FGA']
    predictor_df.loc[0,'FGA']=FGA
    
    FGP=df_team_2021.loc[hometeam,'FG%']
    predictor_df.loc[0,'FG%']=FGP
    
    TFGM=df_team_2021.loc[hometeam,'3FGM']
    predictor_df.loc[0,'3FGM']=TFGM
    
    TFGMA=df_team_2021.loc[hometeam,'3FGA']
    predictor_df.loc[0,'3FGA']=TFGMA
    
    TFGP=df_team_2021.loc[hometeam,'3FG%']
    predictor_df.loc[0,'3FG%']=TFGP
    
    FTM=df_team_2021.loc[hometeam,'FTM']
    predictor_df.loc[0,'FTM']=FTM
    
    FTA=df_team_2021.loc[hometeam,'FTA']
    predictor_df.loc[0,'FTA']=FTA
    
    FTP=df_team_2021.loc[hometeam,'FT%']
    predictor_df.loc[0,'FT%']=FTP
    
    OFF=df_team_2021.loc[hometeam,'OFF']
    predictor_df.loc[0,'OFF']=OFF
    
    DEF=df_team_2021.loc[hometeam,'DEF']
    predictor_df.loc[0,'DEF']=DEF
    
    REB=df_team_2021.loc[hometeam,'REB']
    predictor_df.loc[0,'REB']=REB
    
    AST=df_team_2021.loc[hometeam,'AST']
    predictor_df.loc[0,'AST']=AST
    
    STL=df_team_2021.loc[hometeam,'STL']
    predictor_df.loc[0,'STL']=STL
    
    BLK=df_team_2021.loc[hometeam,'BLK']
    predictor_df.loc[0,'BLK']=BLK
    
    TO=df_team_2021.loc[hometeam,'TO']
    predictor_df.loc[0,'TO']=TO
    
    PF=df_team_2021.loc[hometeam,'PF']
    predictor_df.loc[0,'PF']=PF
    
    PTS=df_team_2021.loc[hometeam,'PTS']
    predictor_df.loc[0,'PTS']=PTS
    
    OPFGM=df_team_2021.loc[awayteam,'FGM']
    predictor_df.loc[0,'OPFGM']=OPFGM
    
    OPFGA=df_team_2021.loc[awayteam,'FGA']
    predictor_df.loc[0,'OPFGA']=OPFGA
    
    OPFGP=df_team_2021.loc[awayteam,'FG%']
    predictor_df.loc[0,'OPFG%']=OPFGP
    
    OP3FGM=df_team_2021.loc[awayteam,'3FGM']
    predictor_df.loc[0,'OP3FGM']=OP3FGM
    
    OP3FGMA=df_team_2021.loc[awayteam,'3FGA']
    predictor_df.loc[0,'OP3FGA']=OP3FGMA
    
    OP3FGP=df_team_2021.loc[awayteam,'3FG%']
    predictor_df.loc[0,'OP3FG%']=OP3FGP
    
    OPFTM=df_team_2021.loc[awayteam,'FTM']
    predictor_df.loc[0,'OPFTM']=OPFTM
    
    OPFTA=df_team_2021.loc[awayteam,'FTA']
    predictor_df.loc[0,'OPFTA']=OPFTA
    
    OPFTP=df_team_2021.loc[awayteam,'FT%']
    predictor_df.loc[0,'OPFT%']=OPFTP
    
    OPOFF=df_team_2021.loc[awayteam,'OFF']
    predictor_df.loc[0,'OPOFF']=OPOFF
    
    OPDEF=df_team_2021.loc[awayteam,'DEF']
    predictor_df.loc[0,'OPDEF']=OPDEF
    
    OPREB=df_team_2021.loc[awayteam,'REB']
    predictor_df.loc[0,'OPREB']=OPREB

    OPAST=df_team_2021.loc[awayteam,'AST']
    predictor_df.loc[0,'OPAST']=OPAST

    OPSTL=df_team_2021.loc[awayteam,'STL']
    predictor_df.loc[0,'OPSTL']=OPSTL
    
    OPBLK=df_team_2021.loc[awayteam,'BLK']
    predictor_df.loc[0,'OPBLK']=OPBLK
    
    OPTO=df_team_2021.loc[awayteam,'TO']
    predictor_df.loc[0,'OPTO']=OPTO
    
    OPPF=df_team_2021.loc[awayteam,'PF']
    predictor_df.loc[0,'OPPF']=OPPF
    
    OPPTS=df_team_2021.loc[awayteam,'PTS']
    predictor_df.loc[0,'OPPTS']=OPPTS
    

DB=predictor_df


def Output():
    DB=predictor_df
    predictor_df['FGM'] = pd.to_numeric(predictor_df['FGM'])
    DB['FGA'] = pd.to_numeric(DB['FGA'])
    DB['FG%'] = pd.to_numeric(DB['FG%'])
    DB['3FGM'] = pd.to_numeric(DB['3FGM'])
    DB['3FGA'] = pd.to_numeric(DB['3FGA'])
    DB['FTM'] = pd.to_numeric(DB['FTM'])
    DB['FT%'] = pd.to_numeric(DB['FT%'])
    DB['OFF'] = pd.to_numeric(DB['OFF'])
    DB['DEF'] = pd.to_numeric(DB['DEF'])
    DB['REB'] = pd.to_numeric(DB['REB'])
    DB['AST'] = pd.to_numeric(DB['AST'])
    DB['STL'] = pd.to_numeric(DB['STL'])
    DB['BLK'] = pd.to_numeric(DB['BLK'])
    DB['TO'] = pd.to_numeric(DB['TO'])
    DB['PF'] = pd.to_numeric(DB['PF'])
    DB['PTS'] = pd.to_numeric(DB['PTS'])
    DB['OPFGM'] = pd.to_numeric(DB['OPFGM'])
    DB['OPFGA'] = pd.to_numeric(DB['OPFGA'])
    DB['OPFG%'] = pd.to_numeric(DB['OPFG%'])
    DB['OP3FGM'] = pd.to_numeric(DB['OP3FGM'])
    DB['OP3FGA'] = pd.to_numeric(DB['OP3FGA'])
    DB['OPFTM'] = pd.to_numeric(DB['OPFTM'])
    DB['OPFT%'] = pd.to_numeric(DB['OPFT%'])
    DB['OPOFF'] = pd.to_numeric(DB['OPOFF'])
    DB['OPDEF'] = pd.to_numeric(DB['OPDEF'])
    DB['OPREB'] = pd.to_numeric(DB['OPREB'])
    DB['OPAST'] = pd.to_numeric(DB['OPAST'])
    DB['OPSTL'] = pd.to_numeric(DB['OPSTL'])
    DB['OPBLK'] = pd.to_numeric(DB['OPBLK'])
    DB['OPTO'] = pd.to_numeric(DB['OPTO'])
    DB['OPPF'] = pd.to_numeric(DB['OPPF'])
    DB['OPPTS'] = pd.to_numeric(DB['OPPTS'])
    

    DB=predictor_df
    Output = print(DB)


    #output1=df_2020_forest_model.predict(DB)
    #if output >0:
        #result = 'Away Team Win'
    #elif output <0:
        #result = 'Home Team Win'


predict = Button(root, text='Predict', font = ('comic sans',15), command=Output)
predict.grid(row = 5)




root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\william\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'FGM'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\william\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-332-bcf2149d326a>", line 165, in Output
    predictor_df['FGM'] = pd.to_numeric(predictor_df['FGM'])
  File "C:\Users\william\Anaconda3\lib

In [279]:
predictor_df = pd.DataFrame()
def action():
    global DB
    import pandas as pd
    predictor_df = pd.DataFrame(columns=['HTeam','VTeam','Betting Line','FGM','FGA','FG%','3FGM',
                                         '3FGA','3FG%','FTM','FTA','FT%','OFF','DEF','REB','AST',
                                         'STL','BLK','TO','PF','PTS','OPFGM','OPFGA','OPFG%',
                                         'OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%','OPOFF',
                                         'OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF','OPPTS'])
    #predictor_df.loc[0,'HTeam']=hometeam
    #predictor_df.loc[0,'VTeam']=awayteam
    #predictor_df.loc[0,'Betting Line']=bettingline
    
    FGM=df_team_2021.loc[hometeam,'FGM']
    predictor_df.loc[0,'FGM']=FGM
    
    FGA=df_team_2021.loc[hometeam,'FGA']
    predictor_df.loc[0,'FGA']=FGA
    
    FGP=df_team_2021.loc[hometeam,'FG%']
    predictor_df.loc[0,'FG%']=FGP
    
    TFGM=df_team_2021.loc[hometeam,'3FGM']
    predictor_df.loc[0,'3FGM']=TFGM
    
    TFGMA=df_team_2021.loc[hometeam,'3FGA']
    predictor_df.loc[0,'3FGA']=TFGMA
    
    TFGP=df_team_2021.loc[hometeam,'3FG%']
    predictor_df.loc[0,'3FG%']=TFGP
    
    FTM=df_team_2021.loc[hometeam,'FTM']
    predictor_df.loc[0,'FTM']=FTM
    
    FTA=df_team_2021.loc[hometeam,'FTA']
    predictor_df.loc[0,'FTA']=FTA
    
    FTP=df_team_2021.loc[hometeam,'FT%']
    predictor_df.loc[0,'FT%']=FTP
    
    OFF=df_team_2021.loc[hometeam,'OFF']
    predictor_df.loc[0,'OFF']=OFF
    
    DEF=df_team_2021.loc[hometeam,'DEF']
    predictor_df.loc[0,'DEF']=DEF
    
    REB=df_team_2021.loc[hometeam,'REB']
    predictor_df.loc[0,'REB']=REB
    
    AST=df_team_2021.loc[hometeam,'AST']
    predictor_df.loc[0,'AST']=AST
    
    STL=df_team_2021.loc[hometeam,'STL']
    predictor_df.loc[0,'STL']=STL
    
    BLK=df_team_2021.loc[hometeam,'BLK']
    predictor_df.loc[0,'BLK']=BLK
    
    TO=df_team_2021.loc[hometeam,'TO']
    predictor_df.loc[0,'TO']=TO
    
    PF=df_team_2021.loc[hometeam,'PF']
    predictor_df.loc[0,'PF']=PF
    
    PTS=df_team_2021.loc[hometeam,'PTS']
    predictor_df.loc[0,'PTS']=PTS
    
    OPFGM=df_team_2021.loc[awayteam,'FGM']
    predictor_df.loc[0,'OPFGM']=OPFGM
    
    OPFGA=df_team_2021.loc[awayteam,'FGA']
    predictor_df.loc[0,'OPFGA']=OPFGA
    
    OPFGP=df_team_2021.loc[awayteam,'FG%']
    predictor_df.loc[0,'OPFG%']=OPFGP
    
    OP3FGM=df_team_2021.loc[awayteam,'3FGM']
    predictor_df.loc[0,'OP3FGM']=OP3FGM
    
    OP3FGMA=df_team_2021.loc[awayteam,'3FGA']
    predictor_df.loc[0,'OP3FGA']=OP3FGMA
    
    OP3FGP=df_team_2021.loc[awayteam,'3FG%']
    predictor_df.loc[0,'OP3FG%']=OP3FGP
    
    OPFTM=df_team_2021.loc[awayteam,'FTM']
    predictor_df.loc[0,'OPFTM']=OPFTM
    
    OPFTA=df_team_2021.loc[awayteam,'FTA']
    predictor_df.loc[0,'OPFTA']=OPFTA
    
    OPFTP=df_team_2021.loc[awayteam,'FT%']
    predictor_df.loc[0,'OPFT%']=OPFTP
    
    OPOFF=df_team_2021.loc[awayteam,'OFF']
    predictor_df.loc[0,'OPOFF']=OPOFF
    
    OPDEF=df_team_2021.loc[awayteam,'DEF']
    predictor_df.loc[0,'OPDEF']=OPDEF
    
    OPREB=df_team_2021.loc[awayteam,'REB']
    predictor_df.loc[0,'OPREB']=OPREB

    OPAST=df_team_2021.loc[awayteam,'AST']
    predictor_df.loc[0,'OPAST']=OPAST

    OPSTL=df_team_2021.loc[awayteam,'STL']
    predictor_df.loc[0,'OPSTL']=OPSTL
    
    OPBLK=df_team_2021.loc[awayteam,'BLK']
    predictor_df.loc[0,'OPBLK']=OPBLK
    
    OPTO=df_team_2021.loc[awayteam,'TO']
    predictor_df.loc[0,'OPTO']=OPTO
    
    OPPF=df_team_2021.loc[awayteam,'PF']
    predictor_df.loc[0,'OPPF']=OPPF
    
    OPPTS=df_team_2021.loc[awayteam,'PTS']
    predictor_df.loc[0,'OPPTS']=OPPTS
    

DB=predictor_df


def Output():
    DB['FGM'] = pd.to_numeric(DB['FGM'])
    DB['FGA'] = pd.to_numeric(DB['FGA'])
    DB['FG%'] = pd.to_numeric(DB['FG%'])
    DB['3FGM'] = pd.to_numeric(DB['3FGM'])
    DB['3FGA'] = pd.to_numeric(DB['3FGA'])
    DB['FTM'] = pd.to_numeric(DB['FTM'])
    DB['FT%'] = pd.to_numeric(DB['FT%'])
    DB['OFF'] = pd.to_numeric(DB['OFF'])
    DB['DEF'] = pd.to_numeric(DB['DEF'])
    DB['REB'] = pd.to_numeric(DB['REB'])
    DB['AST'] = pd.to_numeric(DB['AST'])
    DB['STL'] = pd.to_numeric(DB['STL'])
    DB['BLK'] = pd.to_numeric(DB['BLK'])
    DB['TO'] = pd.to_numeric(DB['TO'])
    DB['PF'] = pd.to_numeric(DB['PF'])
    DB['PTS'] = pd.to_numeric(DB['PTS'])
    DB['OPFGM'] = pd.to_numeric(DB['OPFGM'])
    DB['OPFGA'] = pd.to_numeric(DB['OPFGA'])
    DB['OPFG%'] = pd.to_numeric(DB['OPFG%'])
    DB['OP3FGM'] = pd.to_numeric(DB['OP3FGM'])
    DB['OP3FGA'] = pd.to_numeric(DB['OP3FGA'])
    DB['OPFTM'] = pd.to_numeric(DB['OPFTM'])
    DB['OPFT%'] = pd.to_numeric(DB['OPFT%'])
    DB['OPOFF'] = pd.to_numeric(DB['OPOFF'])
    DB['OPDEF'] = pd.to_numeric(DB['OPDEF'])
    DB['OPREB'] = pd.to_numeric(DB['OPREB'])
    DB['OPAST'] = pd.to_numeric(DB['OPAST'])
    DB['OPSTL'] = pd.to_numeric(DB['OPSTL'])
    DB['OPBLK'] = pd.to_numeric(DB['OPBLK'])
    DB['OPTO'] = pd.to_numeric(DB['OPTO'])
    DB['OPPF'] = pd.to_numeric(DB['OPPF'])
    DB['OPPTS'] = pd.to_numeric(DB['OPPTS'])
    
DB.head()
    
print(predictor_df.shape)
DB=predictor_df

(0, 0)


In [266]:
def Output():
    DB['FGM'] = pd.to_numeric(DB['FGM'])
    DB['FGA'] = pd.to_numeric(DB['FGA'])
    DB['FG%'] = pd.to_numeric(DB['FG%'])
    DB['3FGM'] = pd.to_numeric(DB['3FGM'])
    DB['3FGA'] = pd.to_numeric(DB['3FGA'])
    DB['FTM'] = pd.to_numeric(DB['FTM'])
    DB['FT%'] = pd.to_numeric(DB['FT%'])
    DB['OFF'] = pd.to_numeric(DB['OFF'])
    DB['DEF'] = pd.to_numeric(DB['DEF'])
    DB['REB'] = pd.to_numeric(DB['REB'])
    DB['AST'] = pd.to_numeric(DB['AST'])
    DB['STL'] = pd.to_numeric(DB['STL'])
    DB['BLK'] = pd.to_numeric(DB['BLK'])
    DB['TO'] = pd.to_numeric(DB['TO'])
    DB['PF'] = pd.to_numeric(DB['PF'])
    DB['PTS'] = pd.to_numeric(DB['PTS'])
    DB['OPFGM'] = pd.to_numeric(DB['OPFGM'])
    DB['OPFGA'] = pd.to_numeric(DB['OPFGA'])
    DB['OPFG%'] = pd.to_numeric(DB['OPFG%'])
    DB['OP3FGM'] = pd.to_numeric(DB['OP3FGM'])
    DB['OP3FGA'] = pd.to_numeric(DB['OP3FGA'])
    DB['OPFTM'] = pd.to_numeric(DB['OPFTM'])
    DB['OPFT%'] = pd.to_numeric(DB['OPFT%'])
    DB['OPOFF'] = pd.to_numeric(DB['OPOFF'])
    DB['OPDEF'] = pd.to_numeric(DB['OPDEF'])
    DB['OPREB'] = pd.to_numeric(DB['OPREB'])
    DB['OPAST'] = pd.to_numeric(DB['OPAST'])
    DB['OPSTL'] = pd.to_numeric(DB['OPSTL'])
    DB['OPBLK'] = pd.to_numeric(DB['OPBLK'])
    DB['OPTO'] = pd.to_numeric(DB['OPTO'])
    DB['OPPF'] = pd.to_numeric(DB['OPPF'])
    DB['OPPTS'] = pd.to_numeric(DB['OPPTS'])
 
    output=df_2020_forest_model.predict(DB)
    if output >0:
        result = 'Away Team Win'
    elif output <0:
        result = 'Home Team Win'


In [323]:
predictor_df = pd.DataFrame()
DB=predictor_df

In [325]:
predictor_df = pd.DataFrame()

hometeam='ATL'
wayteam = 'BOS'

bettingline = 150

FGM=df_team_2021.loc[hometeam,'FGM']
predictor_df.loc[0,'FGM']=FGM
    
FGA=df_team_2021.loc[hometeam,'FGA']
predictor_df.loc[0,'FGA']=FGA
    
FGP=df_team_2021.loc[hometeam,'FG%']
predictor_df.loc[0,'FG%']=FGP
    
TFGM=df_team_2021.loc[hometeam,'3FGM']
predictor_df.loc[0,'3FGM']=TFGM
    
TFGMA=df_team_2021.loc[hometeam,'3FGA']
predictor_df.loc[0,'3FGA']=TFGMA
    
TFGP=df_team_2021.loc[hometeam,'3FG%']
predictor_df.loc[0,'3FG%']=TFGP
    
FTM=df_team_2021.loc[hometeam,'FTM']
predictor_df.loc[0,'FTM']=FTM
    
FTA=df_team_2021.loc[hometeam,'FTA']
predictor_df.loc[0,'FTA']=FTA
    
FTP=df_team_2021.loc[hometeam,'FT%']
predictor_df.loc[0,'FT%']=FTP
    
OFF=df_team_2021.loc[hometeam,'OFF']
predictor_df.loc[0,'OFF']=OFF
    
DEF=df_team_2021.loc[hometeam,'DEF']
predictor_df.loc[0,'DEF']=DEF
    
REB=df_team_2021.loc[hometeam,'REB']
predictor_df.loc[0,'REB']=REB
    
AST=df_team_2021.loc[hometeam,'AST']
predictor_df.loc[0,'AST']=AST
    
STL=df_team_2021.loc[hometeam,'STL']
predictor_df.loc[0,'STL']=STL
    
BLK=df_team_2021.loc[hometeam,'BLK']
predictor_df.loc[0,'BLK']=BLK
    
TO=df_team_2021.loc[hometeam,'TO']
predictor_df.loc[0,'TO']=TO
    
PF=df_team_2021.loc[hometeam,'PF']
predictor_df.loc[0,'PF']=PF
    
PTS=df_team_2021.loc[hometeam,'PTS']
predictor_df.loc[0,'PTS']=PTS
    
OPFGM=df_team_2021.loc[awayteam,'FGM']
predictor_df.loc[0,'OPFGM']=OPFGM
    
OPFGA=df_team_2021.loc[awayteam,'FGA']
predictor_df.loc[0,'OPFGA']=OPFGA
    
OPFGP=df_team_2021.loc[awayteam,'FG%']
predictor_df.loc[0,'OPFG%']=OPFGP
    
OP3FGM=df_team_2021.loc[awayteam,'3FGM']
predictor_df.loc[0,'OP3FGM']=OP3FGM
    
OP3FGMA=df_team_2021.loc[awayteam,'3FGA']
predictor_df.loc[0,'OP3FGA']=OP3FGMA
    
OP3FGP=df_team_2021.loc[awayteam,'3FG%']
predictor_df.loc[0,'OP3FG%']=OP3FGP
    
OPFTM=df_team_2021.loc[awayteam,'FTM']
predictor_df.loc[0,'OPFTM']=OPFTM
    
OPFTA=df_team_2021.loc[awayteam,'FTA']
predictor_df.loc[0,'OPFTA']=OPFTA
    
OPFTP=df_team_2021.loc[awayteam,'FT%']
predictor_df.loc[0,'OPFT%']=OPFTP
    
OPOFF=df_team_2021.loc[awayteam,'OFF']
predictor_df.loc[0,'OPOFF']=OPOFF
    
OPDEF=df_team_2021.loc[awayteam,'DEF']
predictor_df.loc[0,'OPDEF']=OPDEF
    
OPREB=df_team_2021.loc[awayteam,'REB']
predictor_df.loc[0,'OPREB']=OPREB

OPAST=df_team_2021.loc[awayteam,'AST']
predictor_df.loc[0,'OPAST']=OPAST

OPSTL=df_team_2021.loc[awayteam,'STL']
predictor_df.loc[0,'OPSTL']=OPSTL
    
OPBLK=df_team_2021.loc[awayteam,'BLK']
predictor_df.loc[0,'OPBLK']=OPBLK
    
OPTO=df_team_2021.loc[awayteam,'TO']
predictor_df.loc[0,'OPTO']=OPTO
    
OPPF=df_team_2021.loc[awayteam,'PF']
predictor_df.loc[0,'OPPF']=OPPF
    
OPPTS=df_team_2021.loc[awayteam,'PTS']
predictor_df.loc[0,'OPPTS']=OPPTS
    
predictor_df.loc[0,'H Betting Line']=bettingline

DB=predictor_df


def Output():
    DB['FGM'] = pd.to_numeric(DB['FGM'])
    DB['FGA'] = pd.to_numeric(DB['FGA'])
    DB['FG%'] = pd.to_numeric(DB['FG%'])
    DB['3FGM'] = pd.to_numeric(DB['3FGM'])
    DB['3FGA'] = pd.to_numeric(DB['3FGA'])
    DB['FTM'] = pd.to_numeric(DB['FTM'])
    DB['FT%'] = pd.to_numeric(DB['FT%'])
    DB['OFF'] = pd.to_numeric(DB['OFF'])
    DB['DEF'] = pd.to_numeric(DB['DEF'])
    DB['REB'] = pd.to_numeric(DB['REB'])
    DB['AST'] = pd.to_numeric(DB['AST'])
    DB['STL'] = pd.to_numeric(DB['STL'])
    DB['BLK'] = pd.to_numeric(DB['BLK'])
    DB['TO'] = pd.to_numeric(DB['TO'])
    DB['PF'] = pd.to_numeric(DB['PF'])
    DB['PTS'] = pd.to_numeric(DB['PTS'])
    DB['OPFGM'] = pd.to_numeric(DB['OPFGM'])
    DB['OPFGA'] = pd.to_numeric(DB['OPFGA'])
    DB['OPFG%'] = pd.to_numeric(DB['OPFG%'])
    DB['OP3FGM'] = pd.to_numeric(DB['OP3FGM'])
    DB['OP3FGA'] = pd.to_numeric(DB['OP3FGA'])
    DB['OPFTM'] = pd.to_numeric(DB['OPFTM'])
    DB['OPFT%'] = pd.to_numeric(DB['OPFT%'])
    DB['OPOFF'] = pd.to_numeric(DB['OPOFF'])
    DB['OPDEF'] = pd.to_numeric(DB['OPDEF'])
    DB['OPREB'] = pd.to_numeric(DB['OPREB'])
    DB['OPAST'] = pd.to_numeric(DB['OPAST'])
    DB['OPSTL'] = pd.to_numeric(DB['OPSTL'])
    DB['OPBLK'] = pd.to_numeric(DB['OPBLK'])
    DB['OPTO'] = pd.to_numeric(DB['OPTO'])
    DB['OPPF'] = pd.to_numeric(DB['OPPF'])
    DB['OPPTS'] = pd.to_numeric(DB['OPPTS'])
    DB['H Betting Line'] = pd.to_numeric(DB['H Betting Line'])
DB.head()

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,40.8,87.2,0.47,12.4,33.4,0.37,19.7,24.2,0.81,10.6,...,10.6,33.6,44.3,23.5,7.7,5.3,14.1,20.4,112.6,150.0


In [326]:
DB.head()

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,40.8,87.2,0.47,12.4,33.4,0.37,19.7,24.2,0.81,10.6,...,10.6,33.6,44.3,23.5,7.7,5.3,14.1,20.4,112.6,150.0


In [314]:
df_2021_test_X.head()

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,39.22,83.74,0.47,12.94,36.19,0.36,16.68,21.11,0.79,8.04,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
1,41.08,87.32,0.47,13.81,38.11,0.36,16.47,21.17,0.78,9.13,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
2,41.46,88.90,0.47,13.60,36.36,0.37,16.11,20.78,0.78,10.63,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
3,41.29,91.08,0.45,15.72,40.82,0.39,17.81,21.64,0.82,10.64,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
4,43.17,90.93,0.47,10.18,29.00,0.35,20.13,26.17,0.77,9.68,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0


In [331]:
output=df_2020_forest_model.predict(DB)
if output >0:
    result = 'Away Team Win'
elif output <0:
    result = 'Home Team Win'
print(result)

Home Team Win
